In [9]:
import networkx as nx
import metis
G = metis.example_networkx()
(edgecuts, parts) = metis.part_graph(G, 3)
colors = ['red','blue','green']
for i, p in enumerate(parts):
    G.node[i]['color'] = colors[p]

nx.write_dot(G, 'example.dot') # Requires pydot or pygraphviz

SyntaxError: invalid syntax (<ipython-input-9-4ec2e7aa9e26>, line 2)